In [1]:
import os
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
###### THIS SECTION IS FOR PURELY UPDATING THE DATA TO MOST RECENT DATA ######
# variables
DOWNLOADS_PATH = 'C:\\Users\\riger\\Downloads'
SCRIPT_PATH = os.getcwd()
# Correct path construction
NHL_CSV_2024_PATH = os.path.join(SCRIPT_PATH, 'generated_data', 'updated_nhl_data_2024.csv')
GOALIE_2024_CSV_PATH = os.path.join(SCRIPT_PATH, 'generated_data', 'updated_goalie_2024_data.csv')


curr_date = pd.to_datetime('today').strftime('%Y-%m-%d')
yesterday = pd.to_datetime('today') - pd.DateOffset(days=1)

# base: https://www.nhl.com/stats/teams?aggregate=0&report=daysbetweengames&reportType=game&dateFrom=2024-10-04&dateTo=2025-01-27&gameType=2&sort=a_gameDate&page=15&pageSize=100
# goalie: https://www.nhl.com/stats/goalies?aggregate=0&reportType=game&dateFrom=2024-10-04&dateTo=2025-01-27&gameType=2&sort=a_gameDate&page=16&pageSize=100

BASE_URL_2024 = 'https://www.nhl.com/stats/teams?aggregate=0&report=daysbetweengames&reportType=game&dateFrom=2024-10-04&dateTo={yesterday}&gameType=2&sort=a_gameDate&page=16&pageSize=100'
GOALIE_URL_2024 = 'https://www.nhl.com/stats/goalies?aggregate=0&reportType=game&dateFrom=2024-10-04&dateTo={yesterday}&gameType=2&sort=a_gameDate&page={}&pageSize=100'

In [3]:
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': DOWNLOADS_PATH,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

In [4]:
# get the current date
current_date = pd.to_datetime('today').date()
# get the day before
yesterday = current_date - pd.DateOffset(days=1)
yesterday_formatted = yesterday.strftime('%Y-%m-%d')
yesterday_formatted

'2025-02-02'

In [5]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

try:
    combined_data = pd.DataFrame()

    # url = BASE_URL_2024.format(1)
    # driver.get(url)
    driver.get(BASE_URL_2024)

    # Wait for the export link to be present and clickable
    export_link = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
    )

    # Scroll to the element
    driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
    export_link.click()
    time.sleep(5)  # Wait for the file to download

    # Process the downloaded file
    downloaded_file = os.path.join(DOWNLOADS_PATH, 'Days between Games.xlsx')
    if os.path.exists(downloaded_file):
        data = pd.read_excel(downloaded_file)
        combined_data = pd.concat([combined_data, data], ignore_index=True)
        os.remove(downloaded_file)  # Remove the downloaded file after processing

    # Save the combined data as a CSV file
    if os.path.exists('Days between Games.xlsx'):
        os.remove('Days between Games.xlsx')
    combined_data.to_csv(NHL_CSV_2024_PATH, index=False)
    print(f"Combined CSV saved at: {NHL_CSV_2024_PATH}")

finally:
    driver.quit()


Combined CSV saved at: c:\Users\riger\Desktop\dataScraper\NHL_data_scraper\generated_data\updated_nhl_data_2024.csv


In [6]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

yesterdayDate = (pd.to_datetime('today') - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
GOALIE_URL_2024 = 'https://www.nhl.com/stats/goalies?aggregate=0&reportType=game&dateFrom=2024-10-04&dateTo={}&gameType=2&sort=a_gameDate&page={}&pageSize=100'

try:
    combined_data = pd.DataFrame()

    # Loop through pages 14 and 15
    for page in [16, 17]:
        # Format the URL with the correct date and page number
        url = GOALIE_URL_2024.format(yesterdayDate, page)
        driver.get(url)

        # Wait for the export link to be present and clickable
        export_link = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
        )

        # Scroll to the element and click the export link
        driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
        export_link.click()
        time.sleep(5)  # Wait for the file to download

        # Process the downloaded file
        downloaded_file = os.path.join(DOWNLOADS_PATH, 'Summary.xlsx')
        if os.path.exists(downloaded_file):
            data = pd.read_excel(downloaded_file)
            combined_data = pd.concat([combined_data, data], ignore_index=True)
            os.remove(downloaded_file)  # Remove the downloaded file after processing

    # Save the combined data as a CSV file
    if os.path.exists('Summary.xlsx'):
        os.remove('Summary.xlsx')
    combined_data.to_csv(GOALIE_2024_CSV_PATH, index=False)
    print(f"Combined CSV saved at: {GOALIE_2024_CSV_PATH}")

finally:
    driver.quit()

Combined CSV saved at: c:\Users\riger\Desktop\dataScraper\NHL_data_scraper\generated_data\updated_goalie_2024_data.csv


In [7]:
hockey_ref_url_2024 = 'https://www.hockey-reference.com/leagues/NHL_2025_games.html'
hockey_ref_dfs = pd.read_html(hockey_ref_url_2024)
hockey_ref_dfs[0].to_csv('./generated_data/nhl_ref_data_2024.csv', index=False)
nhl_ref_data_2024 = pd.read_csv('./generated_data/nhl_ref_data_2024.csv')

In [8]:
nhl_data_2024 = pd.read_csv('./generated_data/updated_nhl_data_2024.csv')
nhl_data_2024 = nhl_data_2024.sort_values(by="Game Date")

# Changing all Montréal Canadiens to Montreal Canadiens in 2024 data
nhl_data_2024 = nhl_data_2024.replace('Montréal Canadiens', 'Montreal Canadiens')

def add_home_column(final_df, temp_df):
    # step 1: loop thorugh the temp df
    # step 2: get display a second df from the main where the dates match the temp df
    # step 3: loop through the second df until 'Team' matches 'Home' and 'Date' matches 'Date' in the temp df
    # step 4: add the 'Home' column to the main df for that row with the value of 1
    for i in range(len(temp_df)):  # Iterate over rows in temp_df
        temp_date = temp_df['Date'][i]
        new_df = final_df[final_df['Game Date'] == temp_date]
        
        for j in range(len(new_df)):
            if new_df['Team'].iloc[j] == temp_df['Home'].iloc[i]:
                if 'Venue' not in final_df.columns:
                    final_df['Venue'] = 'Away'
                
                # Update the specific row in the original df
                final_df.loc[new_df.index[j], 'Venue'] = 'Home'
    
    return final_df

final_nhl_data_2024 = add_home_column(nhl_data_2024, nhl_ref_data_2024)
final_nhl_data_2024 = final_nhl_data_2024.dropna()

In [9]:
def add_time_column(final_df, temp_df):
    # Ensure 'Time' column exists in final_df
    if 'Time' not in final_df.columns:
        final_df['Time'] = None

    # Iterate over rows in temp_df
    for i in range(len(temp_df)):
        temp_time = temp_df['Time'].iloc[i]
        temp_date = temp_df['Date'].iloc[i]
        temp_home = temp_df['Home'].iloc[i]
        temp_visitor = temp_df['Visitor'].iloc[i]

        # Filter final_df for matching date
        matching_rows = final_df[final_df['Game Date'] == temp_date]

        for j in range(len(matching_rows)):
            team = matching_rows['Team'].iloc[j]

            # Check if the team matches either Home or Visitor in temp_df
            if team == temp_home or team == temp_visitor:
                # Update the 'Time' column in the original final_df
                final_df.loc[matching_rows.index[j], 'Time'] = temp_time

    return final_df

final_nhl_data_2024 = add_time_column(nhl_data_2024, nhl_ref_data_2024)

In [10]:
final_nhl_data_2024['Time'] = final_nhl_data_2024['Time'].apply(
    lambda x: pd.to_datetime(x, format='%I:%M %p').strftime('%H:%M') if pd.notnull(x) else None
)

final_nhl_data_2024 = final_nhl_data_2024.sort_values(by="Game Date")

final_nhl_data_2024.rename(columns={'OT':'OT Losses','GD/GP':'Net Goals','Shots/GP':'Shots For','SA/GP':'Shots Against','SD/GP':'Shot Diff'}, inplace=True)
final_nhl_data_2024['Result'] = final_nhl_data_2024.apply(lambda row: 'W' if row['Net Goals'] > 0 else 'L', axis=1)

final_nhl_data_2024.tail()

,Team,Opp Team,Game Date,Days Btwn Games,GP,W,L,T,OT Losses,P,...,Shots Against,Shot Diff,PP Opp/GP,TS/GP,PP%,PK%,FOW%,Venue,Time,Result
77,Vancouver Canucks,DET,2025-02-02,1,1,0,0,--,1,1,...,18,9,0,1,--,100,59.3,Home,20:00,L
76,Colorado Avalanche,PHI,2025-02-02,1,1,1,0,--,0,2,...,24,8,2,1,50,100,63.0,Home,15:00,W
73,Detroit Red Wings,VAN,2025-02-02,0,1,1,0,--,0,2,...,27,-9,1,0,0,--,40.7,Away,20:00,W
74,St. Louis Blues,UTA,2025-02-02,1,1,1,0,--,0,2,...,25,4,2,3,0,100,62.5,Away,19:00,W
83,Utah Hockey Club,STL,2025-02-02,1,1,0,1,--,0,0,...,29,-4,3,2,0,100,37.5,Home,19:00,L


In [11]:
nhl_team_abbr = {
    'ANA' : 'Anaheim Ducks',
    'BOS' : 'Boston Bruins',
    'BUF' : 'Buffalo Sabres',
    'CAR' : 'Carolina Hurricanes',
    'CBJ' : 'Columbus Blue Jackets',
    'CGY' : 'Calgary Flames',
    'CHI' : 'Chicago Blackhawks',
    'COL' : 'Colorado Avalanche',
    'DAL' : 'Dallas Stars',
    'DET' : 'Detroit Red Wings',
    'EDM' : 'Edmonton Oilers',
    'FLA' : 'Florida Panthers',
    'LAK' : 'Los Angeles Kings',
    'MIN' : 'Minnesota Wild',
    'MTL' : 'Montreal Canadiens',
    'NJD' : 'New Jersey Devils',
    'NSH' : 'Nashville Predators',
    'NYI' : 'New York Islanders',
    'NYR' : 'New York Rangers',
    'OTT' : 'Ottawa Senators',
    'PHI' : 'Philadelphia Flyers',
    'PIT' : 'Pittsburgh Penguins',
    'SJS' : 'San Jose Sharks',
    'SEA' : 'Seattle Kraken',
    'STL' : 'St. Louis Blues',
    'TBL' : 'Tampa Bay Lightning',
    'TOR' : 'Toronto Maple Leafs',
    'UTA' : 'Utah Hockey Club',
    'VAN' : 'Vancouver Canucks',
    'VGK' : 'Vegas Golden Knights',
    'WPG' : 'Winnipeg Jets',
    'WSH' : 'Washington Capitals',
    'ARI' : 'Utah Hockey Club'
}

In [12]:
goalie_df_2024 = pd.read_csv('./generated_data/updated_goalie_2024_data.csv')

In [13]:
goalie_df_2024['Team'] = goalie_df_2024['Team'].apply(lambda abbr: nhl_team_abbr.get(abbr, abbr))

final_nhl_data_2024 = final_nhl_data_2024.merge(goalie_df_2024[['Team', 'Game Date', 'Player', 'Sv%']], 
                                      left_on=['Team', 'Game Date'], 
                                      right_on=['Team', 'Game Date'], 
                                      how='left')

final_nhl_data_2024['goalie'] = final_nhl_data_2024['Player']
final_nhl_data_2024.drop(columns=['Player'], inplace=True)

In [14]:
final_nhl_data_2024['Opp Team'] = final_nhl_data_2024['Opp Team'].apply(lambda abbr: nhl_team_abbr.get(abbr, abbr))

In [15]:
final_nhl_data_2024.drop(columns=['W', 'L', 'T', 'GP', 'OT Losses', 'P', 'P%'], inplace=True)
final_nhl_data_2024.columns = [x.lower() for x in final_nhl_data_2024.columns]

In [16]:
final_nhl_data_2024 = final_nhl_data_2024.rename(columns={"game date": "date", "opp team": "opponent", "days btwn games": "rest days"})

In [17]:
new_order = ['date', 'time', 'venue', 'rest days', 'result', 'gf/gp', 'ga/gp', 'net goals', 'opponent', 'shots for', 'shots against', 'shot diff', 'pp opp/gp', 'ts/gp', 'pp%',
       'pk%', 'fow%', 'team', 'goalie', 'sv%']
final_nhl_data_2024 = final_nhl_data_2024[new_order]

In [18]:
# filter by only entries from the last day already existing in the data until yesterday
old_df = pd.read_csv('final_nhl_data_2024.csv')
last_date_in_old_df = old_df['date'].iloc[-1]
last_date_in_old_df

'2025-01-27'

In [19]:
new_data = final_nhl_data_2024[final_nhl_data_2024['date'] > last_date_in_old_df]
# new_data = new_data[new_data['date'] <= yesterday]

new_data.head()

,date,time,venue,rest days,result,gf/gp,ga/gp,net goals,opponent,shots for,shots against,shot diff,pp opp/gp,ts/gp,pp%,pk%,fow%,team,goalie,sv%
0,2025-01-28,19:00,Home,2,L,1,4,-3,Chicago Blackhawks,35,21,14,3,2,0,50,59.3,Tampa Bay Lightning,Jonas Johansson,0.850
1,2025-01-28,21:00,Away,2,W,3,1,2,Calgary Flames,23,33,-10,4,5,25,80,50.0,Washington Capitals,Logan Thompson,0.970
2,2025-01-28,19:00,Away,1,W,4,1,3,Tampa Bay Lightning,21,35,-14,2,3,50,100,40.7,Chicago Blackhawks,Arvid Soderblom,0.971
3,2025-01-28,21:00,Home,1,L,1,3,-2,Washington Capitals,33,23,10,5,4,20,75,50.0,Calgary Flames,Dustin Wolf,0.870
4,2025-01-28,19:30,Away,1,L,2,5,-3,New York Islanders,32,28,4,2,0,0,--,53.9,Colorado Avalanche,Mackenzie Blackwood,0.852


In [20]:
# append the new data to the old data
final_nhl_data_2024 = pd.concat([old_df, new_data], ignore_index=True)

final_nhl_data_2024.to_csv('final_nhl_data_2024.csv', index=False)